**Nhóm: A-Team**

**Thành viên:**

*     Vũ Quốc Anh - 20C11003
*     Văn Khải Nguyên - 20C11009
*     Hà Hữu Pháp - 20C11010
*     Nguyễn Trần Duy Minh - 20C11041
*     Phạm Anh Việt - 20C11060

# Tổng quan

- Nhóm sử dụng Transfer learning với pretrained model là EfficientNetB7 do đây là model khá mới, cho độ chính xác cao và dễ dàng cài đặt vì có các thư viện hỗ trợ sẵn.

- Thông tin chi tiết về mô hình EfficientNet: https://arxiv.org/abs/1905.11946 .

- Kết hợp với pretrained model, nhóm sử dụng thêm kỹ thuật Data augmentation và Test time Augmentation để tăng cường độ chính xác nhận dạng.

- Độ chính xác cuối cùng  vơi điểm private test là 0.97138 và điểm public test là 0.97185.



# 1. Cài đặt môi trường

## 1.1 Cài đặt mạng EfficientNet

In [ ]:
!pip install efficientnet

## 1.2 Import các thư viện cần thiết

In [ ]:
import math, re, os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
import tensorflow as tf
import tensorflow.keras.layers as L
import efficientnet.tfkeras as efn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping

## 1.3 Phát hiện TPU

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

## 1.4 Load dataset từ Google Cloud Service

In [ ]:
# Data access
GCS_DS_PATH = KaggleDatasets().get_gcs_path('plant-pathology-2020-fgvc7')

# Configuration
EPOCHS = 200
BATCH_SIZE = 8 * strategy.num_replicas_in_sync
IM_Z = 768 # kích thước cao rộng của hình

In [ ]:
def format_path(st):
    return GCS_DS_PATH + '/images/' + st + '.jpg'

# 2. Tổng quan về bộ dữ liệu

Bộ dữ liệu được tạo ra với mục đích nhận dạng các loại bệnh trên lá táo, thay cho công việc của chuyên gia, kết quả chia thành 4 loại:
- Healthy
- Rust
- Scab
- Combination


Bộ train: gồm 1821 tấm hình. Số hình của mỗi loại phân bố không đều:
* Lá khỏe mạnh(healthy): 561 
* Lá rust: 
* Lá scab: 
* Lá bị nhiều bệnh kết hợp:

Bộ test: gồm 1821 ảnh. Chưa được gán nhãn

Kích thước từng ảnh: 2048 x 1365


## 2.1 Đọc thông tin bộ dataset
* Số lượng mẫu bộ train và test
* Số lượng mẫu theo từng class

In [ ]:
train = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/train.csv')
test = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/test.csv')
sub = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/sample_submission.csv')

train_paths = train.image_id.apply(format_path).values
test_paths = test.image_id.apply(format_path).values

N = len(train_paths)
print(f"- Độ dài bộ train: {len(train_paths)}")
print(f"- Độ dài bộ test: {len(test_paths)}")

N_healthy = train['healthy'].mean() * N
N_multiple_diseases = train['multiple_diseases'].mean() * N
N_rust = train['rust'].mean() * N
N_scab = train['scab'].mean() * N

print(f'Healthy: {N_healthy} mẫu')
print(f'Multiple diseases: {N_multiple_diseases} mẫu')
print(f'Rust: {N_rust} mẫu')
print(f'Scab: {N_scab} mẫu')

train.describe()

## 2.2 Chia bộ train và bộ validation và tách nhãn thành one-hot vector

Bộ train và bộ validation được chia theo tỉ lệ 90% : 10%

In [ ]:
# Chia train / validation
# Lấy label theo dạng one-hot vector

train_labels = train.loc[:, 'healthy':].values

print(f"- Train labels:\n {train_labels}")

train_paths, valid_paths, train_labels, valid_labels = train_test_split(
    train_paths, train_labels, test_size=0.3, random_state=2020)

print(f" - Độ dài bộ train: {len(train_paths)}")
print(f" - Độ dài bộ validation: {len(valid_paths)}")

# 3. Cài đặt mô hình

## 3.1 Hàm helper
* decode_image(): load hình và tiền xử lý
  -  Hình ảnh sẽ được scale giá trị còn từ 0 -> 1
  -  Hình ảnh sẽ được resize về kích thước 768 x 768
* data_augment(): Augmentation là kĩ thuật tạo ra dữ liệu training từ dữ liệu mà ta đang có
  - Hình ảnh được lật theo chiều ngang (flip left to right)
  - Sau đó hình ảnh được lật theo chiều dọc ( flip up to down) 

In [ ]:
def decode_image(filename, label=None, image_size=(IM_Z, IM_Z)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label

def data_augment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    if label is None:
        return image
    else:
        return image, label

## 3.2 Chuẩn bị data

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_paths, train_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .cache()
    .map(data_augment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(512)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((valid_paths, valid_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(test_paths)
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
)

## 3.3 Các hàm callback: 
* Hàm thay đổi learning rate động
* Checkpoint callback: Lưu lại mô hình có độ lỗi trên bộ validation thấp nhất

In [ ]:
def build_lrfn(lr_start=0.00001, lr_max=0.000075, 
               lr_min=0.000001, lr_rampup_epochs=20, 
               lr_sustain_epochs=0, lr_exp_decay=.8):
    lr_max = lr_max * strategy.num_replicas_in_sync

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_exp_decay**(epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
        return lr
    
    return lrfn

ch_p = ModelCheckpoint(filepath="model_ef1.h5", monitor='val_loss', save_weights_only=True, save_best_only=True,  verbose=1)

In [ ]:
lrs = []

lrnfn = build_lrfn()

for epoch in range(0, 100):
    lr = lrnfn(epoch)
    lrs.append(lr)

plt.plot(lrs)

## 3.4 Tạo mạng transfer learning dùng pretrained model là EfficientNetB7

Mô hình bao gồm:
-  Mạng pretrained EffcientNetB7 với bộ trọng số của imagenet
-  Lớp Global Average Pooling
-  Lớp Dense 4 neurals cho 4 lớp dùng hàm activation softmax 

Hàm optimize: Hàm Adam

Hàm loss: Categorical Crossentropy


In [ ]:
with strategy.scope():
    model = tf.keras.Sequential([
        efn.EfficientNetB7(
            input_shape=(IM_Z, IM_Z, 3),
            weights='imagenet',
            include_top=False
        ),
        L.GlobalAveragePooling2D(),
        L.Dense(train_labels.shape[1], activation='softmax')
    ])
        
    model.compile(
        optimizer='adam',
        loss = 'categorical_crossentropy',
        metrics=['categorical_accuracy']
    )
    model.summary()

In [ ]:
lrfn = build_lrfn()
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=0)
STEPS_PER_EPOCH = train_labels.shape[0] // BATCH_SIZE 
EarlyStop_callback = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

## 3.5 Train mạng transfer learning

- Mô hình được train với 60 epoch


In [ ]:
history = model.fit(
    train_dataset, 
    epochs=EPOCHS, 
    callbacks=[ch_p, EarlyStop_callback],
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=valid_dataset
)

In [ ]:
# summarize history for accuracy

print(history.history.keys())

plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model categorical accuracy')
plt.ylabel('categorical accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
model.load_weights("model_ef1.h5")

## 3.6 TTA

### 3.6.1 Chuẩn bị bộ TTA

In [ ]:
def dummy_data_augment(image):
    return image
    
data_augments = [dummy_data_augment, tf.image.flip_left_right, tf.image.flip_up_down]

In [ ]:
TTA_test_datasets = [(
    tf.data.Dataset
    .from_tensor_slices(test_paths)
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .map(d_augment, num_parallel_calls=AUTO)
) for d_augment in data_augments]

### 3.6.2  Dự doán theo TTA

In [ ]:
combined_probs_TTAs = [model.predict(dataset) for dataset in TTA_test_datasets]

In [ ]:
probs_TTA = np.zeros(combined_probs_TTAs[0].shape)

for p in combined_probs_TTAs:
    probs_TTA += p
    
probs_TTA = probs_TTA / len(combined_probs_TTAs)
    
len(combined_probs_TTAs)

## 3.7 Lưu kết quả

In [ ]:
submission_name = 'submission_eff_TTA'

for d_aug in data_augments:
    submission_name += "-"
    submission_name += d_aug.__name__

submission_name += ".csv"
submission_name

In [ ]:
sub.loc[:, 'healthy':] = probs_TTA
sub.to_csv(submission_name, index=False)
sub.head()